# VacationPy
----

#### Note
* Had to enable some things (in Bash, after activating Python Data environment, before opening Jupyter):
    - jupyter nbextension enable --py gmaps
    - jupyter nbextension enable --py widgetsnbextension
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
# Read Cities csv that was created in the WeatherPy notebook and store into a Pandas DataFrame
input_data_file = "output_data/cities.csv"
city_df = pd.read_csv(input_data_file)

# Drop the extra index column
city_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

city_df.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,carnarvon,-24.8667,113.6333,77.00,73,1,5.75,AU,1611959521
1,tessalit,20.1986,1.0114,63.84,25,0,13.29,ML,1611959521
2,phonhong,18.5007,102.4166,61.29,76,27,2.21,LA,1611959521
3,hobart,-42.8794,147.3294,69.01,77,20,5.75,AU,1611959402
4,ust-tsilma,65.4410,52.1498,-2.67,90,100,9.42,RU,1611959522


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [24]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = city_df[["Latitude","Longitude"]]
humidity = city_df["Humidity"].astype(float)

# create smaller set of cities to test with - so as not to waste as many API calls
mini_locations = locations.iloc[0:100]
mini_humidity = humidity.iloc[0:100]
mini_locations.head()

print(max(mini_humidity))

100.0


In [22]:
mini_humidity.head()

,Humidity
0,73
1,25
2,76
3,77
4,90


In [23]:
print(max(mini_humidity))

Humidity


In [19]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(mini_locations, weights=mini_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add Heatmap layer to map
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# Plot the heatmap
fig = gmaps.figure(layout=figure_layout)

# Create the heat layer




### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [67]:
hotel_df.describe()

,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,1.030000e+02
mean,-2.344937,-1.778760,76.180000,62.504854,17.145631,8.809709,1.611960e+09
std,19.270021,79.255065,5.133003,23.251344,17.054933,4.871452,7.584394e+01
min,-42.879400,-159.775000,66.130000,15.000000,0.000000,1.140000,1.611959e+09
25%,-20.323200,-69.011800,72.635000,51.500000,0.000000,5.325000,1.611960e+09
50%,0.383300,-2.100000,77.000000,71.000000,14.000000,8.050000,1.611960e+09
75%,13.485300,50.030050,80.600000,79.500000,40.000000,12.595000,1.611960e+09
max,33.306200,176.700000,84.990000,94.000000,49.000000,19.570000,1.611960e+09


In [73]:
city_df.describe()

,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,553.000000,553.000000,553.000000,553.000000,553.000000,553.000000,5.530000e+02
mean,20.227134,18.275414,46.838608,73.117541,52.189873,7.918590,1.611960e+09
std,33.846989,89.443727,34.589713,19.306517,38.848439,5.513955,7.719259e+01
min,-54.800000,-179.166700,-51.790000,15.000000,0.000000,0.250000,1.611959e+09
25%,-8.118100,-59.516700,25.070000,65.000000,10.000000,3.440000,1.611960e+09
50%,22.285500,24.662200,59.740000,78.000000,61.000000,6.910000,1.611960e+09
75%,50.033400,97.968300,75.200000,87.000000,90.000000,11.300000,1.611960e+09
max,78.218600,179.316700,93.200000,100.000000,100.000000,34.520000,1.611960e+09


In [84]:
# Turn these columns into floats so we can do numeric comparisons on them
city_df[["Max Temp", "Humidity", "Cloudiness","Wind Speed"]].astype(float)

# Narrow down the cities to fit weather conditions.
perfect_temp = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 70)]
low_clouds = perfect_temp.loc[(perfect_temp["Cloudiness"] == 0)]
not_too_windy = low_clouds.loc[(low_clouds["Wind Speed"] < 10)]

# Drop any rows with null values ad Store into variable named hotel_df.

hotel_df = not_too_windy.dropna().set_index("City").reset_index()


# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df.head(100)


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,mar del plata,-38.0023,-57.5575,75.20,57,0,3.44,AR,1611959529,
1,saint-pierre,-21.3393,55.4781,78.80,65,0,5.75,RE,1611959556,
2,ler,8.3018,30.1418,78.51,15,0,8.59,SS,1611959575,
3,caravelas,-17.7125,-39.2481,77.32,81,0,7.23,BR,1611959592,
4,saint-philippe,-21.3585,55.7679,78.80,65,0,5.75,RE,1611959600,
5,leo,11.1000,-2.1000,75.63,18,0,7.96,BF,1611959618,
6,bentiu,9.2333,29.8333,75.51,16,0,8.10,SS,1611959624,
7,araouane,18.9048,-3.5265,70.03,21,0,9.22,ML,1611959628,
8,cap malheureux,-19.9842,57.6142,77.00,82,0,3.00,MU,1611959469,
9,tahoua,14.8888,5.2692,71.60,23,0,6.91,NE,1611959635,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [91]:
# Set parameters to do a nearby places search for hotels witin 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}
# Iterate through the df of cities with perfect weather and hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():

    # get latitude & longitude of each city
    lat = row["Latitude"]
    lng = row["Longitude"]
       
    # change the location parameter for each city
    params['location'] = f"{lat},{lng}"

    # assemble the url and call the nearby places search API
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index,"Hotel Name"] = (response["results"][0]["name"])
    except: 
        pass
    
# Remove all of the rows from the hotel_df for cities that the API did not find hotels for
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]

ler
....has no hotel
araouane
....has no hotel
tenenkou
....has no hotel


In [95]:
hotel_df.head(10)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,mar del plata,-38.0023,-57.5575,75.20,57,0,3.44,AR,1611959529,Hotel Las Rocas
1,saint-pierre,-21.3393,55.4781,78.80,65,0,5.75,RE,1611959556,"Villa Delisle, Hotel & Spa"
3,caravelas,-17.7125,-39.2481,77.32,81,0,7.23,BR,1611959592,Pousada Liberdade
4,saint-philippe,-21.3585,55.7679,78.80,65,0,5.75,RE,1611959600,Les Embruns Du Baril
5,leo,11.1000,-2.1000,75.63,18,0,7.96,BF,1611959618,Hotel Espoir
6,bentiu,9.2333,29.8333,75.51,16,0,8.10,SS,1611959624,Mandela hôtel
8,cap malheureux,-19.9842,57.6142,77.00,82,0,3.00,MU,1611959469,Azure Beach Boutique Hotel
9,tahoua,14.8888,5.2692,71.60,23,0,6.91,NE,1611959635,Hotel Etab
11,saint-joseph,-21.3667,55.6167,78.80,65,0,5.75,RE,1611959672,"""Plantation Bed and Breakfast"


In [98]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [100]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, 
                            info_box_content = hotel_info)

# Add the layers to the map
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))